### PTR motifs
Eraslan et al find some additional PTR motifs. Do they add explanatory power?

In [3]:
from importlib import reload
import re
import random
from pathlib import Path
random.seed(1337)
import os
import pickle
from decimal import Decimal
import collections

import numpy as np
np.random.seed(1337)
import pandas as pd
pd.options.mode.chained_assignment = None 
import scipy.stats as stats

from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

# Dont use GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import tensorflow as tf
import keras
from keras import backend as K
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import model
import utils
import utils_data

Using TensorFlow backend.


In [2]:
with open(Path("../Data/data_dict.pkl"), 'rb') as handle:
    data_dict = pickle.load(handle)

In [3]:
library_fn = utils_data.LibraryEncoder("library", {"egfp_unmod_1":0, "random":1})
utr_model_combined = load_model(Path("../Models/utr_model_combined_residual.h5"), custom_objects={'FrameSliceLayer': model.FrameSliceLayer})

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [7]:
# compute predictions
df = data_dict["ptr"]
one_hot_fn = utils_data.OneHotEncoder("utr")
generator = utils_data.DataSequence(df, encoding_functions=[one_hot_fn, library_fn], 
                                                shuffle=False)
pred = utr_model_combined.predict_generator(generator, verbose=0)

In [44]:
X_without = pred
y = np.array(df["ptr"])[:,np.newaxis]
reg_without = LinearRegression().fit(X_without, y)
r2_without = reg_without.score(X_without, y)
print(r2_without)
print(np.sqrt(r2_without))

0.029128003158193194
0.17066928006584312


In [27]:
eraslan_5utr_motifs = ["AACUU", "ACCUGC", "AGCAAC", "AGCCCCC", "AGCGGAA", "CACGU", "CAGAC", "CCCACCC",
                      "CCGUGGG", "CCUUGGA", "CGGAAG", "CUCUGAG", "CUCUUUC", "CUGGGAGC", "CUGUCCU",
                      "GAUAC", "GCCGCC", "GGCGCCCG", "GUGGGAA", "UACAGG", "UCGAC", "UCUGGGA", "UGACCU",
                      "UUCCG"]
motif_counter = utils_data.MotifExtractor("utr", "", eraslan_5utr_motifs)
motif_counts = motif_counter.__call__(df)
motif_counts = np.transpose(np.concatenate([x[:,np.newaxis] for x in motif_counts], axis=1))

In [31]:
X = np.concatenate([pred, motif_counts], axis=1)

In [46]:
X = np.concatenate([pred, motif_counts], axis=1)
r2 = reg.score(X, y)
r2_adjusted = utils.adjust_r2(r2, X.shape[0], X.shape[1])
print(r2)
print(np.sqrt(r2))
print(r2_adjusted)
print(np.sqrt(r2_adjusted))

0.03963109636097517
0.19907560463546298
0.037552195798937316
0.19378388942050193
